In [1]:
import sys
sys.path.append("../")

## initial Outreach Pipeline

In [2]:
import os
import re
import pandas as pd
import sqlite3

from src.utils.helpers import (
    get_actual_date,
    url_to_names,
    post_interaction_mapper,
    likes_reactions_repost_mapper,
    outrch_commnty_mangmnt_mapper,
)
from datetime import datetime

In [3]:
data_lake_path = "./data/data_lake/"
data_warehouse_path = "./data/data_warehouse/data_warehouse.db"

all_raw_files = os.listdir(data_lake_path)
print(all_raw_files)

['raw_new_accounts_comments.csv', 'raw_new_accounts_posts.csv', 'raw_new_accounts_reactions.csv']


In [4]:
def create_dataset(data_lake_path) -> pd.DataFrame:
    df = pd.DataFrame()
    for file in all_raw_files:
        temp_df = pd.read_csv(data_lake_path+file)
        print(f"{file} (shape): {temp_df.shape}")
        df = pd.concat([df, temp_df])
    return df

def check_null_values(df: pd.DataFrame):
    null_count = df.isnull().sum()
    check_null_columns = ["likeCount", "commentCount", "repostCount", "action"]
    for column_name in check_null_columns:
        if null_count[column_name] != 0:
            print(f"log(missing_values):{column_name}: {null_count[column_name]}")
            df = df[df[column_name].notna()]
    return df

def fix_data_types(df: pd.DataFrame):
    df["postTimestamp"] = pd.to_datetime(df["postTimestamp"]).dt.tz_localize(None)
    return df

def remove_unwanted_columns(df: pd.DataFrame):
    column_names = ['imgUrl', 'type', 'postDate', 'commentContent', 'commentUrl',
                'sharedPostUrl', 'timestamp',
                'videoUrl', 'sharedJobUrl']
    return df.drop(columns=column_names, axis=1)

def _word_has_numbers(word):
    return any(char.isdigit() for char in word)

def create_profile_url_to_name_mapper(df: pd.DataFrame):
    mapper = {}
    profile_urls = df["profileUrl"].unique()
    for url in profile_urls:
        raw_name = url.split("/")[4]
        name_split = raw_name.split("-")
        if len(name_split) > 1:
            name = " ".join(name_split[:-1])
        mapper[url] = name
        
    return mapper
    
def get_profile_names(df: pd.DataFrame):
    mapper = create_profile_url_to_name_mapper(df=df)
    df["profileName"] = df["profileUrl"].map(mapper)
    return df

def get_actions_data(df: pd.DataFrame) -> pd.DataFrame:
    df["activity"] = df.action.tolist()
    df["action"] = df["activity"].apply(post_interaction_mapper)
    df["action"] = df.action.map(likes_reactions_repost_mapper)
    df = df.drop("activity", axis=1)
    return df

def reorder_columns(df: pd.DataFrame):
    return df.reindex(columns=['profileUrl', 'profileName', 'postTimestamp', 
                       "postContent", "postUrl", "action", "likeCount", 
                        "commentCount", "repostCount"])

def sqlite_connection(data_warehouse_path: str):
    conn = sqlite3.connect(data_warehouse_path)
    return conn
def cut_post_size(text):
    return str(text)[:100]

def post_fix(df:pd.DataFrame):
    df["postContent"] = df["postContent"].apply(cut_post_size)
    return df
    
def create_data_warehouse(data_warehouse_path: str):
    file = open(data_warehouse_path, 'w')
    file.close()
         
    conn = sqlite_connection(data_warehouse_path=data_warehouse_path)
    cursor = conn.cursor()
    
    create_query = """
    CREATE TABLE profile_data 
    (ID INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    profile_url varchar(225) NOT NULL, 
    profile_name varchar(225) NOT NULL,  
    post_time datetime NOT NULL, 
    postContent varchar(225) NOT NULL, 
    post_url varchar(225), 
    action varchar(225) NOT NULL, 
    likes integer NOT NULL, 
    comments integer NOT NULL, 
    reposts NOT NULL);"""
    
    cursor.execute(create_query)
    conn.close()
def add_data_to_warehouse(df: pd.DataFrame, data_warehouse_path: str):
    data = list(df.values)
    conn = sqlite_connection(data_warehouse_path=data_warehouse_path)
    cur = conn.cursor()
    insert_query = """
    INSERT INTO profile_data 
    VALUES(
    NULL,
    ?, ?, ?, ?, ?, ?, ?, ?, ?
    )
    """
    cur.executemany(insert_query, data)
    conn.commit()
    conn.close()

In [5]:
df = create_dataset(data_lake_path=data_lake_path)
df = check_null_values(df=df)
df = remove_unwanted_columns(df=df)
df = fix_data_types(df=df)
df = get_profile_names(df=df)
df = post_fix(df=df)
df = get_actions_data(df=df)
df = reorder_columns(df=df)
df = df = df[df["postTimestamp"] > "2023.01.01"].sort_values("postTimestamp")
df["postTimestamp"] = df["postTimestamp"].apply(lambda x: str(x.replace(microsecond=0)))

create_data_warehouse(data_warehouse_path=data_warehouse_path)
add_data_to_warehouse(df=df, data_warehouse_path=data_warehouse_path)

raw_new_accounts_comments.csv (shape): (130, 17)
raw_new_accounts_posts.csv (shape): (94, 15)
raw_new_accounts_reactions.csv (shape): (1049, 17)


In [7]:
x = pd.read_sql_query("SELECT * FROM profile_data", sqlite_connection(data_warehouse_path=data_warehouse_path))

In [9]:
x.head()

,ID,profile_url,profile_name,post_time,postContent,post_url,action,likes,comments,reposts
0,1,https://www.linkedin.com/in/jasmin-rummer-1571...,jasmin rummer,2023-01-01 08:31:46,Excited to start a new challenge as Head of Ta...,https://www.linkedin.com/feed/update/urn:li:ac...,post,170,38,0
1,2,https://www.linkedin.com/in/jasmin-rummer-1571...,jasmin rummer,2023-01-01 16:56:48,Ich habe einen neuen Job und arbeite jetzt als...,https://www.linkedin.com/feed/update/urn:li:ac...,comments,115,22,0
2,3,https://www.linkedin.com/in/jasmin-rummer-1571...,jasmin rummer,2023-01-02 12:47:50,It's time for a #change . Today I'm starting a...,https://www.linkedin.com/feed/update/urn:li:ac...,comments,342,66,0
3,4,https://www.linkedin.com/in/jasmin-rummer-1571...,jasmin rummer,2023-01-02 20:06:12,👋🏼 Goodbye Uniper 💙🙌 Hello Brenntag 💜I am prou...,https://www.linkedin.com/feed/update/urn:li:ac...,comments,250,61,0
4,5,https://www.linkedin.com/in/jasmin-rummer-1571...,jasmin rummer,2023-01-03 10:32:11,So darf das Jahr gerne beginnen: Mit dem Unter...,https://www.linkedin.com/feed/update/urn:li:ac...,comments,56,3,0


## Extra stuff

In [25]:
df["month"] = pd.to_datetime(df["postTimestamp"]).dt.month_name()

In [31]:
posts = (
        df.groupby(["profileName", "month", "action"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

In [32]:
df.action.unique()

array(['post', 'comments', 'replied to a comment', 'likes and reactions',
       'liked a comment on their own post', 'repost'], dtype=object)

In [33]:
posts = posts.pivot_table(
        index="profileName",
        columns="month",
        values=[
            'post', 'repost'
        ],
        fill_value=0,
    )
desired_order = ["January", "Febraury", "March",
                 "April", "May", "June" ,
                 "July", "August", "September", "October"]

posts = posts.reindex(columns=desired_order, level=1)

In [34]:
outreach = (
        df.groupby(["profileName", "month", "action"])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    )

In [35]:
outreach = outreach.pivot_table(
        index="profileName",
        columns="month",
        values=[
            'comments', 'replied to a comment', 
            'likes and reactions','liked a comment on their own post'
        ],
        fill_value=0,
    )
desired_order = ["January", "Febraury", "March",
                 "April", "May", "June" ,
                 "July", "August", "September", "October"]

outreach = outreach.reindex(columns=desired_order, level=1)

In [38]:
excel_writer = pd.ExcelWriter(
        f"new_profiles_{datetime.now().year}_CM_outreach.xlsx",
        engine="xlsxwriter",
    )
posts.to_excel(excel_writer, sheet_name="posts")
outreach.to_excel(excel_writer, sheet_name="cm_outreach")
excel_writer.close()

In [39]:
!open .